In [1]:
 import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
iris = datasets.load_iris() 
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [0]:
X = iris.data
y = iris.target

In [0]:
scaler_X = StandardScaler()
X = scaler_X.fit_transform(X)

In [5]:
onehot = OneHotEncoder(categories= 'auto')
y.shape

(150,)

In [6]:
y = y.reshape(-1, 1)
y.shape

(150, 1)

In [0]:
y = onehot.fit_transform(y).toarray()


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [9]:
X_train.shape

(105, 4)

In [10]:
X_test.shape

(45, 4)

In [11]:
input_neurons = X.shape[1]
input_neurons

4

In [12]:
hidden_neurons = int(np.ceil((X.shape[1] + y.shape[1]) /2))
hidden_neurons

4

In [13]:
output_neurons = y.shape[1]
output_neurons

3

In [0]:
weights = {'hidden': tf.Variable(tf.random_normal([input_neurons, hidden_neurons])),
          'output': tf.Variable(tf.random_normal([hidden_neurons, output_neurons]))}

In [0]:
bias = {'hidden': tf.Variable(tf.random_normal([hidden_neurons])),
        'output': tf.Variable(tf.random_normal([output_neurons]))}

In [0]:
X_placeholder = tf.placeholder('float', [None, input_neurons])
y_placeholder = tf.placeholder('float', [None, output_neurons])

In [0]:
def placeholder_model(X, weights, bias):
  hidden_layer = tf.add(tf.matmul(X, weights['hidden']), bias['hidden'])
  hidden_layer_activation = tf.nn.relu(hidden_layer)
  output_layer = tf.add(tf.matmul(hidden_layer_activation, weights['output']), bias['output'])
  return output_layer

In [0]:
model = placeholder_model(X_placeholder, weights, bias)

In [0]:
error = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = model, labels = y_placeholder))

In [0]:
optimizer = tf.train.AdadeltaOptimizer(learning_rate=0.01).minimize(error)

In [0]:
batch_size = 8
batch_total = int(len(X_train,)/batch_size)

In [32]:
batch_total

13

In [0]:
X_batches = np.array_split(X_train, batch_total)

In [37]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for epochs in range(3000):
    average_error = 0.0
    batch_total = int(len(X_train)/ batch_size)
    X_batches = np.array_split(X_train, batch_total)
    y_batches = np.array_split(y_train, batch_total)
    for i in range(batch_total):
      X_batch, y_batch = X_batches[i], y_batches[i]
      _, coast = sess.run([optimizer, error], feed_dict={X_placeholder: X_batch, y_placeholder: y_batch})
      average_error += coast/batch_total
    if epochs % 500 == 0:
      print('Epocas: ' + str((epochs + 1)) + ' error: ' + str(average_error))
  weights_final, bias_final = sess.run([weights, bias])


Epocas: 1 error: 2.6666190440838156
Epocas: 501 error: 2.5080469204829288
Epocas: 1001 error: 2.3630137351843024
Epocas: 1501 error: 2.235179515985342
Epocas: 2001 error: 2.1196291675934424
Epocas: 2501 error: 2.012460075891935


In [39]:
weights_final 

{'hidden': array([[-2.000405  ,  0.86991864,  0.34392816, -1.7325704 ],
        [-2.2727716 ,  1.0427396 ,  0.29981467, -0.04445807],
        [-3.0465841 ,  0.7579779 , -0.30680946,  0.08437754],
        [-0.15225857, -0.3887925 ,  1.1062188 , -0.053211  ]],
       dtype=float32),
 'output': array([[ 0.6716419 , -1.5587947 , -0.01094528],
        [-1.4725971 , -0.3257853 , -0.25196978],
        [ 2.4679248 , -0.69876564,  2.3573182 ],
        [-0.6921633 , -0.2190583 , -0.9284067 ]], dtype=float32)}

In [41]:
bias_final

{'hidden': array([ 1.5018016 ,  0.5938379 , -0.22444746, -0.06553368], dtype=float32),
 'output': array([-0.3397646 , -0.41480982,  0.7925989 ], dtype=float32)}

In [0]:
prevision_test = placeholder_model(X_placeholder, weights_final, bias_final)
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  r1 = sess.run(prevision_test, feed_dict={X_placeholder: X_test})
  r2 = sess.run(tf.nn.softmax(r1))
  r3 = sess.run(tf.argmax(r2, 1))
  

In [44]:
r1

array([[ -1.7129974 ,  -1.4063333 ,   1.6267369 ],
       [  1.7510376 ,  -9.00987   ,  -0.64418083],
       [  2.6397665 ,  -7.8479204 ,   0.4773969 ],
       [ -0.23976398,  -6.5463715 ,  -0.42572623],
       [  1.0197929 ,  -2.2170358 ,   1.4101784 ],
       [ -1.1908777 ,  -5.467507  ,  -0.8273869 ],
       [  3.4173217 , -12.944107  ,  -1.2296515 ],
       [  0.8153441 ,  -2.1960788 ,   4.156605  ],
       [  1.4497602 ,  -5.473536  ,   0.29733592],
       [ -1.5260645 ,  -1.338681  ,   1.617842  ],
       [ -0.4975202 ,  -2.267116  ,   0.25669223],
       [ -2.1062236 ,  -1.2316142 ,   1.1526036 ],
       [ -2.733109  ,  -3.1718621 ,   0.21886927],
       [ -0.23265481,  -1.9725739 ,   3.269407  ],
       [  4.828042  , -15.721306  ,  -0.9968559 ],
       [  2.0069468 ,  -6.014201  ,   0.67561054],
       [ -1.0692508 ,  -1.0874333 ,   1.4625318 ],
       [  1.9415312 ,  -5.709405  ,   0.75542223],
       [ -0.15210122,  -3.0460677 ,  -0.08529967],
       [ -0.0617795 ,  -1.50353

In [46]:
r2

array([[3.27112600e-02, 4.44508307e-02, 9.22837973e-01],
       [9.16444123e-01, 1.94403183e-05, 8.35364237e-02],
       [8.96796525e-01, 2.50005342e-05, 1.03178397e-01],
       [5.45813024e-01, 9.95678594e-04, 4.53191251e-01],
       [3.97323847e-01, 1.56101771e-02, 5.87065995e-01],
       [4.07791942e-01, 5.66398539e-03, 5.86544096e-01],
       [9.90500391e-01, 7.76562246e-08, 9.49948095e-03],
       [3.41250971e-02, 1.67969172e-03, 9.64195192e-01],
       [7.59385288e-01, 7.47675134e-04, 2.39867091e-01],
       [3.93694751e-02, 4.74831276e-02, 9.13147449e-01],
       [3.03366929e-01, 5.16942851e-02, 6.44938827e-01],
       [3.39940153e-02, 8.15155655e-02, 8.84490430e-01],
       [4.81032245e-02, 3.10188811e-02, 9.20877874e-01],
       [2.91041732e-02, 5.10878908e-03, 9.65787053e-01],
       [9.97055650e-01, 1.18645549e-09, 2.94441101e-03],
       [7.90856123e-01, 2.59751017e-04, 2.08884165e-01],
       [6.86912835e-02, 6.74535930e-02, 8.63855064e-01],
       [7.65765429e-01, 3.64197

In [49]:
r3

array([2, 0, 0, 0, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 0, 2,
       0, 2, 2, 0, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 2,
       2])

In [51]:
y_test2 = np.argmax(y_test, 1)
y_test2

array([1, 0, 1, 0, 2, 0, 0, 2, 1, 1, 1, 1, 0, 2, 1, 1, 2, 2, 1, 2, 1, 2,
       0, 2, 0, 1, 2, 1, 0, 1, 1, 0, 0, 2, 1, 2, 1, 0, 1, 2, 0, 2, 1, 2,
       1])